In [1]:
import numpy as np
from scipy.stats import invgamma
from scipy.stats import norm as normal

import plotly.express as px
import pandas as pd

In [2]:
y = [[324, 797, 507, 748, 457, 381],
     [180, 130, 101, 314, 182, 121, 
      238, 153, 116, 296, 163, 119, 
      235, 138, 113, 192, 132, 104]]
J = len(y)
N = [16, 48]
n = [len(yj) for yj in y]
ybar = [np.mean(yj) for yj in y]

# https://stackoverflow.com/questions/35213592/numpy-calculate-square-of-norm-2-of-vector
sqnorm = lambda x: np.inner(x, x)

In [3]:
rng = np.random.default_rng() # PCG64 generator
n_iter = 1000
n_burn_in = 100

In [4]:
sigma2 =[1, 1]
mu = [0, 0]
Ybar=[0, 0]
c1 = [n[j]/N[j] * ybar[j] for j in range(0,J)]
c2 = [1 - n[j]/N[j] for j in range(0,J)]

sigma2_s, mu_s, Ybar_s =[], [], []
for iter in range(0, n_iter):
    for j in range(0, J):
        mu_j = normal.rvs(loc=ybar[j], scale=np.sqrt(sigma2[j]/n[j]))
        sigma2_j = invgamma.rvs(a=n[j]/2, scale=0.5*sqnorm(y[j] - mu_j))
        
        if (mu_j < 1 or sigma2_j < 1):
            print(f'WARN: reject this sample too close to origin, iter = {iter}')
            continue
        else:
            mu[j], sigma2[j] = mu_j, sigma2_j
        Y_OOS = normal.rvs(loc=mu[j], scale=np.sqrt(sigma2[j]/(N[j]-n[j])))
        Ybar[j] = c1[j] + c2[j]*Y_OOS
    Ybar_a = np.sum([N[j] * Ybar[j] for j in range(0,J)]) / np.sum(N)
    
    if iter > n_burn_in:
        sigma2_s.append(sigma2.copy())
        mu_s.append(mu.copy())
        Ybar_s.append(Ybar.copy())

In [5]:
px.scatter(pd.DataFrame(mu_s, columns={'<i>μ<sub>1</sub></i>', '<i>μ<sub>2</sub></i>'}),
           title="trace <i>μ</i>").show("notebook_connected")
px.scatter(pd.DataFrame(sigma2_s, columns={'<i>σ<sup>2</sup><sub>1</sub></i>', '<i>σ<sup>2</sup><sub>2</sub></i>'}),
           title="trace <i>σ<sup>2</sup></i>").show("notebook_connected")
px.scatter(pd.DataFrame(Ybar_s, columns={r'$\bar{Y_1}$', r'$\bar{Y_2}$'}),
           title=r"$\text{trace}\ \bar{Y_{1,2}}$").show("notebook_connected")

In [6]:
N_mat = np.matrix(N) 
N_mat = N_mat / np.sum(N_mat)
Ybar_a_s = np.matrix(Ybar_s) * N_mat.transpose()
df = pd.DataFrame(np.hstack((np.matrix(Ybar_s),Ybar_a_s)), columns=["stratum 1", "stratum 2", "all"])

In [7]:
pd.DataFrame((df.mean().rename("mean"), 
              df.median().rename("median"), 
              df.quantile(0.25/100).rename("2.5%-tile"), 
              df.quantile(97.5/100).rename("97.5%-tile")))

,stratum 1,stratum 2,all
mean,537.640750,168.084717,260.473725
median,537.209892,168.073205,260.015873
2.5%-tile,266.422000,126.804875,190.871177
97.5%-tile,712.337969,193.188109,305.880438
